<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Gráficos e visualização

No final desta lição, você aprenderá:

1. Como usar a conveniente interface de plotagem do xarray, construída sobre [matplotlib](https://matplotlib.org/), para visualizar seus conjuntos de dados;
2. E também como `hvplot` fornece uma interface igualmente conveniente para gráficos baseados em [bokeh](https://docs.bokeh.org/en/latest/index.html).

## Conteúdo

1. [Basic plotting](#basic)
1. [Histograms](#hist)
1. [2D plots](#2d)
1. [1D line plots](#1d)
1. [Faceting or multiple subplots](#facet)
1. [Geography: matplotlib and cartopy](#geo)
1. [Interactive bokeh plots using hvplot](#bokeh)


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
xr.set_options(display_style="html")

## Carregar dados

Primeiro, vamos carregar um conjunto de dados de tutorial para visualizar:


In [ ]:
ds = xr.tutorial.open_dataset("air_temperature.nc").rename({"air": "Tair"})

# vamos adicionar um campo gradiente com os atributos apropriados
ds["dTdx"] = ds.Tair.differentiate("lon") / 110e3 / np.cos(ds.lat * np.pi / 180)
ds["dTdy"] = ds.Tair.differentiate("lat") / 105e3
ds.dTdx.attrs = {"long_name": "$∂T/∂x$", "units": "°C/m"}
ds.dTdy.attrs = {"long_name": "$∂T/∂y$", "units": "°C/m"}

ds

Esse conjunto de dados tem três "variáveis de dados", `Tair` é a temperatura do ar e` dTdx`
e `dTdy` são gradientes horizontais desse campo de temperatura. Os três "dados
variáveis" são tridimensionais com dimensões `(tempo, lat, lon)`.


<a id='basic'></a>

## Plotagem básica: .plot()

Os objetos DataArray têm um método `plot`. Este método cria gráficos usando
`matplotlib`, então todo o seu conhecimento matplotlib existente será útil aqui (por exemplo, [Gráficos com qualidade de publicação em Python com Matplotlib](https://www.fschuch.com/blog/2020/10/14/graficos-com-qualidade-de-publicacao-em-python-com-matplotlib/)).

Por padrão `.plot ()` produz:

1. Um gráfico de linha para dados 1-D usando `plt.plot()`;
2. Um gráfico `pcolormesh` para dados 2-D usando` plt.pcolormesh()`;
3. Um histograma para tudo o mais usando `plt.hist()`.


<a id='hist'></a>

## Histogramas

`Tair` é tridimensional, então temos um histograma dos valores de temperatura. Repare na
descrição no eixo x. Um dos recursos de plotagem convenientes do xarray é que ele
usa os atributos (`attrs`) de `Tair` para rotular adequadamente os eixos e a barras de cores.


In [ ]:
ds.Tair.plot()

Você pode passar argumentos extras para a chamada `hist()` subjacente. Veja a [documentação matplotlib](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.hist.html) para
todos os argumentos de palavra-chave possíveis.

**Dica:** Observe que os valores retornados são exatamente o que matplotlib retornaria.

#### Exercício

Atualize o gráfico acima para mostrar 50 colunas não preenchidas em vez de barras preenchidas.


<a id='2d'></a>

## Plotagens 2D

Agora vamos explorar gráficos 2D. Vamos selecionar um único passo de tempo de `Tair` para
visualizar:


In [ ]:
ds.Tair.isel(time=1).plot()

Observe quanta informação há nesse gráfico (para apenas uma linha de código)!

Os eixos `x` e `y` são rotulados com nomes completos - "Latitude", "Longitude" - junto
com as unidades. A barra de cores tem um rótulo legal, novamente com unidades. E o título nos diz o carimbo de data/hora dos dados apresentados.

`plot` recebe muitos argumentos de palavras-chave e é bastante sofisticado (veja
https://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html).

Aqui está uma figura mais complicada que define explicitamente `tempo` como o eixo x,
personaliza a barra de cores e sobrepõe dois contornos em níveis específicos.

**Dica:** Outras opções para gráficos 2D incluem `.plot.contour`,` .plot.contourf`,
`.plot.imshow`.


In [ ]:
ds.Tair.isel(lon=1).plot(
    x="time",  # coordenada para traçar no eixo x do gráfico
    robust=True,  # definir os limites da barra de cores para o 2º e 98º percentil dos dados
    cbar_kwargs={
        "orientation": "horizontal",
        "label": "custom label",
        "pad": 0.2,
    },  # passados para plt.colorbar
)

### Exercício

Atualize o gráfico acima para usar um mapa de cores matplotlib diferente.


### Exercício

Agora sobreponha um gráfico de contorno no topo do gráfico anterior.


<a id='1d'></a>

## Gráficos de linha 1D

Xarray também é capaz de traçar linhas envolvendo `plt.plot()`. Como nos exemplos anteriores, os eixos são rotulados e os argumentos de palavras-chave podem ser passados para o chamada `matplotlib` subjacente.


In [ ]:
ds.Tair.isel(time=1, lon=10).plot(marker="o")

Digamos que queremos comparar gráficos lineares de temperatura em três diferentes
latitudes. Podemos usar o kwarg `hue` para fazer isso.


In [ ]:
ds.Tair.isel(time=1).sel(lat=[40, 50, 60], method="nearest").plot(
    x="lon", hue="lat"
)

## Costumização

Todas as funções de plotagem do xarray usam uma grande lista de `kwargs` que personalizam seu
comportamento. Uma lista completa pode ser vista aqui:
https://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html.

Digamos que xarray não envolve todas as funcionalidades disponíveis em matplotlib.
A estratégia geral para fazer plotagens que são mais complexas que o exemplos acima são

1. Crie um eixo matplotlib `ax`;
2. Use xarray para fazer uma aproximação do gráfico final especificando `Ax = ax`;
3. Use os métodos `ax` para personalizar totalmente o gráfico.


<a id='facet'></a>

## Facetar

Facetar (*faceting*) é a arte de apresentar "pequenos múltiplos" dos dados. É uma
maneira eficaz de visualizar variações de dados 3D onde as fatias 2D são
visualizado em um painel (subplot) e a terceira dimensão é variada entre
painéis.

É aqui que o xarray realmente aumenta a funcionalidade do matplotlib. Nós vamos usar a média mensal para ilustrar

In [ ]:
monthly_means = ds.groupby("time.month").mean()
# as reduções de agrupamento de xarray dropam atributos. Vamos atribuí-los de volta para obter bons rótulos.
monthly_means.Tair.attrs = ds.Tair.attrs
monthly_means

Observe que as dimensões agora são `lat, lon, month`.

Queremos visualizar como a temperatura média mensal do ar varia com o mês do
o ano.

A maneira mais simples de facetar é especificar os kwargs `row` ou` col` que devem ser um nome de dimensão. Aqui usamos `month` para que cada painel ou a "faceta" do gráfico, que apresenta o campo de temperatura média em um determinado mês. Já que um gráfico de 12 colunas seria muito pequeno para interpretar, podemos "embrulhar" as facetas em múltiplas linhas usando `col_wrap`:

In [ ]:
fg = monthly_means.Tair.plot(
    col="month",
    col_wrap=4,  # cada linha terá no máximo 4 colunas
)

Todas as personalizações usuais são possíveis:


In [ ]:
fg = monthly_means.Tair.plot(
    col="month",
    col_wrap=4,
    # Os kwargs restantes personalizam o gráfico da mesma forma que vimos antes
    robust=True,
    cmap='RdYlBu_r',
    cbar_kwargs={
        "orientation": "horizontal",
        "shrink": 0.8,
        "aspect": 40,
        "pad": 0.1,
    },
)

O objeto FacetGrid retornado `fg` tem muitas propriedades e métodos úteis, por exemplo:

1. `fg.fig` fornece um identificador para a figura;
2. `fg.axes` é uma matriz de objetos numpy para lidar com cada eixo individualmente;
3. `fg.set_xlabels` e` fg.set_ylabels` podem ser usados para alterar os rótulos dos eixos.

Consulte https://xarray.pydata.org/en/stable/api.html#faceting para obter uma lista completa.

### Exercício

Use esses métodos para definir um título para a figura usando `suptitle`, bem como
mude os rótulos dos eixos x e y.


In [ ]:
fg

### Modificando todas as facetas de uma figura

O objeto FacetGrid tem alguns métodos mais avançados que permitem personalizar ainda mais as figuras.

Aqui nós ilustramos o uso de `map` e` map_dataarray` que permitem mapear
plotagem de funções para um `FacetGrid` existente. As funções passadas para `map` e
`map_dataarray` deve ter uma assinatura particular. Veja a docstring para mais
detalhes.

Alternativamente, pode-se fazer um loop em `fg.axes` e modificar cada subtrama individual como
necessário:


In [ ]:
fg = monthly_means.Tair.plot(col="month", col_wrap=4)

# Use isto para traçar contornos em cada painel
# Observe que esta chamada de plotagem usa os gradientes DataArray originais
fg.map_dataarray(
    xr.plot.contour, x="lon", y="lat", colors="k", levels=13, add_colorbar=False
)

# Adicione um ponto (ou qualquer outra coisa!)
fg.map(lambda: plt.plot(250, 40, markersize=20, marker=".", color="w"))

### Dica para os painéis: use a facetação para plotar múltiplos DataArrays

A facetação pode ser usada para plotar múltiplos DataArrays em um Dataset. O truque é
usar `to_array()` para converter um Dataset em um DataArray e então plotar.

Esse truque só funciona quando é sensato usar o mesmo mapa e escala de cores para todos os DataArrays como com `dTdx` e `dTdy`:


In [ ]:
gradients = monthly_means[["dTdx", "dTdy"]].to_array("gradient")
gradients

In [ ]:
fg = gradients.isel(month=slice(None, None, 3)).plot.contourf(
    levels=13,
    col="month",
    row="gradient",
    robust=True,
    cmap=mpl.cm.coolwarm,
    cbar_kwargs={
        "orientation": "horizontal",
        "shrink": 0.8,
        "aspect": 40,
        "label": "Gradient [°C/m]",
    },
)

<a id="geo"></a>

## Geografia: Matplotlib e Cartopy

Uma vez que a funcionalidade de plotagem padrão do xarray se baseia em matplotlib, podemos perfeitamente usar `Cartopy` para fazer mapas legais:

1. Especifique uma `projeção` para o gráfico ao criar uma nova figura `fig` com eixo `axis`;
2. Peça explicitamente ao xarray para plotar para o eixo `axis` passando o kwarg `ax = axis`;
3. Especifique a projeção dos dados usando `transform` (`PlateCarree` aqui) em `.plot ()`.


In [ ]:
import cartopy.crs as ccrs

fig, axis = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.Orthographic(-90, 30))
)

ds.Tair.isel(time=1).plot(
    ax=axis,
    transform=ccrs.PlateCarree(),  # Isso é importante!
    # As opções padrões que vimos antes
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.7},
    robust=True,
)
axis.coastlines()  # função cartopy

### Mapas facetados

Podemos fazer mapas facetados. Uma vez que `FacetGrid` cria os eixos para os quais traça, nós
precisa passar o kwarg `projection` em `subplot_kws`. Isso garante que os subplots são configuradas corretamente para *cartopy*.


In [ ]:
fg = monthly_means.Tair.isel(month=[1, 2, 3]).plot(
    col="month",
    transform=ccrs.PlateCarree(),  # Lembre-se de fornecer isso
    subplot_kws={
        "projection": ccrs.LambertConformal(
            central_longitude=-95, central_latitude=45
        )
    },
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.8, "aspect": 40},
    robust=True,
)

# Vamos adicionar uma linha costeira a cada eixo
# ótimo motivo para usar FacetGrid.map
fg.map(lambda: plt.gca().coastlines())

<a id="bokeh"></a>

## Gráficos interativos Bokeh usando hvplot

A funcionalidade de plotagem embutida no Xarray envolve matplotlib.

O ecossistema de plotagem `holoviews` fornece o pacote `hvplot` para permitir
visualização de objetos xarray (e outros) (https://hvplot.holoviz.org/). Estes
plotagens são baseadas em [Bokeh](https://bokeh.org/).

`hvplot` faz uso da interface de acesso do xarray. Isso significa que todos os objetos xarray ganham um atributo `.hvplot` que permite acessar a funcionalidade `hvplot` tão facilmente quanto você usaria `.plot`


In [ ]:
import hvplot.xarray

**`hvplot` faz as mesmas escolhas padrão que xarray**


In [ ]:
# Histograma quando tem mais de duas dimensões
ds.Tair.hvplot()

In [ ]:
# Array 2D produz um gráfico quadmesh
ds.Tair.isel(time=1).hvplot(cmap="fire")

In [ ]:
# 1D produz um gráfico de linha
ds.Tair.isel(time=1, lon=1).hvplot()

### Interatividade

Mas `hvplot` brilha quando a interatividade é usada. Aqui podemos dar a ele _todos_ os
dados e pedir para criar um bom controle deslizante para controlar a fração de tempo usando o
`groupby.

In [ ]:
ds.Tair.hvplot(
    groupby="time",
    clim=(250, 295),  # adds a widget for time  # sets colorbar limits
)

Animações são fáceis


In [ ]:
# definir limites constantes da barra de cores
ds.Tair.hvplot(
    groupby="time",  # adiciona um widget para o tempo
    clim=(250, 295),  # define limites do mapa de cores
    widget_type="scrubber",
    widget_location="bottom",
)

### Geografia


In [ ]:
ds.Tair.isel(time=1).hvplot(
    projection=ccrs.Orthographic(-90, 30),
    coastline=True,
)

## Recursos adicionais

1. Xarray's visualization gallery:
   https://xarray.pydata.org/en/stable/examples/visualization_gallery.html
2. Xarray's plotting documentation:
   https://xarray.pydata.org/en/stable/plotting.html
3. hvplot's plotting documentation:
   https://hvplot.holoviz.org/user_guide/Gridded_Data.html
